<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b4f6009203eebd07ab3e4640c98bd6ac14293779138855a7622b12bf0eff884a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-13 10:42:01
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 29.06 K (0.2%)
Current PnL: -21.16 L (-13.86%)
CY Booked + Current PnL: -7.64 L (-5.0%)
-------------------
Total profit:  2.00 L
Total loss:  -23.16 L
-------------------
Total Booked + Current PnL: 19.34 L (15.4%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.29%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 87.83 L (60.37%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,102.77,53.0,M-SC,2.72,87384.0,-13393.0,13483.0,0.46,-13.29,15.43,0.09,245.0,-0.99,0.61,14.80,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.25,48.0,X-MC,3.35,88755.0,-1923.0,16828.0,0.05,-2.12,18.96,16.44,101.0,-0.11,0.62,17.63,X40,ATH,PHARMA
34,ICICIGI,2252.93,-11.75,62.0,X-MC,7.56,190734.0,16414.0,18787.0,1.10,9.42,9.85,20.19,91.0,0.87,1.32,25.95,X40,ATH,INSURANCE
38,INDIGOPNTS,1408.00,133.26,80.0,M-SC,4.36,152830.0,-21729.0,21763.0,2.49,-12.45,14.24,0.02,221.0,-1.00,1.06,32.94,OX40N,NTT,PAINTS
51,NESTLEIND,1377.00,-2.49,66.0,X-LC,8.75,300245.0,34819.0,21978.0,0.38,13.12,7.32,21.40,11.0,1.58,2.09,20.63,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.01,85.0,X-LC,7.95,252296.0,528.0,52679.0,3.51,0.21,20.88,21.13,27.0,0.01,1.75,35.71,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,133.26,80.0,M-SC,4.36,152830.0,-21729.0,21763.0,2.49,-12.45,14.24,0.02,221.0,-1.00,1.06,32.94,OX40N,NTT,PAINTS
79,VAIBHAVGBL,521.00,76.31,64.0,H-SC,7.67,153013.0,-29762.0,140833.0,2.42,-16.28,92.04,60.77,125.0,-0.21,1.06,37.80,XR,NTT,JEWELLERY
39,INDUSINDBK,1800.00,311.06,81.0,L-MC,6.66,50416.0,-26790.0,52181.0,2.32,-34.70,103.50,32.89,258.0,-0.51,0.35,38.84,XR,NTT,BANKS
60,ROUTE,2227.21,-55.56,48.0,H-SC,22.98,73124.0,-59942.0,151827.0,2.30,-45.05,207.63,69.05,118.0,-0.39,0.51,7.74,SR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-54.55,56.0,H-SC,9.56,261723.0,-24782.0,282111.0,-3.65,-8.65,107.79,89.82,134.0,-0.09,1.82,38.47,XY24,NTT,FINANCE
15,CAMPUS,393.00,-26.26,40.0,M-SC,4.08,153630.0,-14145.0,69994.0,-2.03,-8.43,45.56,33.28,210.0,-0.20,1.07,19.07,XY24,NTT,FOOTWEAR
17,CERA,9475.00,-28.16,30.0,H-SC,2.26,130950.0,-44953.0,86977.0,-1.35,-25.56,66.42,23.89,149.0,-0.52,0.91,13.09,OX40N,NTT,CERAMICS
7,ASIANTILES,137.00,7588.89,66.0,L-SC,15.86,84368.0,-9442.0,86064.0,-1.21,-10.07,102.01,81.67,269.0,-0.11,0.59,63.38,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.15,49.0,H-SC,7.51,91680.0,-6300.0,27907.0,-0.78,-6.43,30.44,22.05,152.0,-0.23,0.64,33.26,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152191.0,1246.0,108208.0,0.10,0.83,71.10,72.51,53.0,0.01,1.06,6.9,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.05,38.0,H-MC,5.89,209453.0,-121.0,74796.0,-0.38,-0.06,35.71,35.63,NaN,-0.00,1.45,16.6,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.0,133.26,80.0,M-SC,4.36,152830.0,-21729.0,21763.0,2.49,-12.45,14.24,0.02,221.0,-1.00,1.06,32.94,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.0,102.77,53.0,M-SC,2.72,87384.0,-13393.0,13483.0,0.46,-13.29,15.43,0.09,245.0,-0.99,0.61,14.80,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.61,51.0,H-SC,2.39,223623.0,-46044.0,82383.0,1.20,-17.07,36.84,13.47,138.0,-0.56,1.55,13.71,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-24.38,49.0,H-MC,6.27,102296.0,-29539.0,71761.0,-0.07,-22.41,70.15,32.03,98.0,-0.41,0.71,14.58,OX40N,NTT,IT
66,SIS,528.0,2023.32,47.0,H-SC,2.14,85680.0,-25352.0,48958.0,-0.27,-22.83,57.14,21.26,156.0,-0.52,0.60,15.76,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152191.0,1246.0,108208.0,0.10,0.83,71.10,72.51,53.0,0.01,1.06,6.90,XR,NTT,IT
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.78,132558.0,9222.0,122404.0,-0.14,7.48,92.34,106.72,119.0,0.08,0.92,32.60,AR,ATH,MISC
61,SAMMAANCAP,326.00,-193.18,57.0,M-SC,36.14,161487.0,12267.0,131919.0,1.39,8.22,81.69,96.62,208.0,0.09,1.12,74.76,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.78,132558.0,9222.0,122404.0,-0.14,7.48,92.34,106.72,119.0,0.08,0.92,32.60,AR,ATH,MISC
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152191.0,1246.0,108208.0,0.10,0.83,71.10,72.51,53.0,0.01,1.06,6.90,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.05,38.0,H-MC,5.89,209453.0,-121.0,74796.0,-0.38,-0.06,35.71,35.63,NaN,-0.00,1.45,16.60,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.34,37.0,M-SC,9.20,113362.0,-1436.0,95473.0,0.43,-1.25,84.22,81.92,223.0,-0.02,0.79,34.20,XR,NTT,DURABLES
47,KPIGREEN,731.05,2.11,43.0,H-SC,10.30,121829.0,-3468.0,62401.0,-0.50,-2.77,51.22,47.03,141.0,-0.06,0.85,52.54,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-17.47,28.0,X-MC,5.77,253650.0,-246.0,82563.0,-0.64,-0.10,32.55,32.42,105.0,-0.00,1.76,14.99,X40,ATH,FMCG
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87117.0,-41553.0,86856.0,-0.75,-32.29,99.70,35.20,219.0,-0.48,0.61,0.26,X40,NTT,FOOTWEAR
1,ABB,7934.00,-43.27,32.0,H-LC,5.15,237216.0,-24403.0,143611.0,0.37,-9.33,60.54,45.57,7.0,-0.17,1.65,0.94,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,32.0,X-MC,1.55,194025.0,-6795.0,74583.0,0.50,-3.38,38.44,33.76,80.0,-0.09,1.35,1.64,X40,ATH,FMCG
54,QUESS,986.00,-52.07,32.0,X-SC,37.24,47821.0,-17185.0,168115.0,-0.47,-26.44,351.55,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.77,44.0,X-LC,0.87,198363.0,-1775.0,22673.0,-0.33,-0.89,11.43,10.44,4.0,-0.08,1.38,4.63,X40,NTT,FMCG
53,PGHH,17907.65,-32.93,32.0,X-MC,1.55,194025.0,-6795.0,74583.0,0.50,-3.38,38.44,33.76,80.0,-0.09,1.35,1.64,X40,ATH,FMCG
65,SIEMENS,4671.50,-3.25,50.0,H-LC,1.61,155645.0,-30450.0,77931.0,1.84,-16.36,50.07,25.51,15.0,-0.39,1.08,14.70,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-7.88,43.0,X-MC,3.22,307209.0,-20782.0,127307.0,0.34,-6.34,41.44,32.48,92.0,-0.16,2.13,4.37,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-6.09,62.0,X-LC,3.28,246201.0,12057.0,36561.0,-0.29,5.15,14.85,20.77,86.0,0.33,1.71,12.42,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,32.0,X-SC,37.24,47821.0,-17185.0,168115.0,-0.47,-26.44,351.55,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87117.0,-41553.0,86856.0,-0.75,-32.29,99.70,35.20,219.0,-0.48,0.61,0.26,X40,NTT,FOOTWEAR
1,ABB,7934.00,-43.27,32.0,H-LC,5.15,237216.0,-24403.0,143611.0,0.37,-9.33,60.54,45.57,7.0,-0.17,1.65,0.94,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,32.0,X-MC,1.55,194025.0,-6795.0,74583.0,0.50,-3.38,38.44,33.76,80.0,-0.09,1.35,1.64,X40,ATH,FMCG
20,COLPAL,3726.84,-6.96,41.0,X-MC,6.97,217300.0,-46065.0,155391.0,-0.20,-17.49,71.51,41.51,84.0,-0.30,1.51,1.97,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,32.0,X-SC,37.24,47821.0,-17185.0,168115.0,-0.47,-26.44,351.55,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.27,45.0,X-SC,3.56,81489.0,-63671.0,139599.0,-0.26,-43.86,171.31,52.31,136.0,-0.46,0.57,9.08,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87117.0,-41553.0,86856.0,-0.75,-32.29,99.70,35.20,219.0,-0.48,0.61,0.26,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.25,48.0,X-MC,3.35,88755.0,-1923.0,16828.0,0.05,-2.12,18.96,16.44,101.0,-0.11,0.62,17.63,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.23,37.0,X-SC,0.97,106695.0,-21243.0,68381.0,0.20,-16.60,64.09,36.84,143.0,-0.31,0.74,9.23,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,62.0,X-LC,8.77,293280.0,-52676.0,119365.0,-0.38,-15.23,40.70,19.28,1.0,-0.44,2.04,8.42,X40,ATH,IT
40,INFY,2275.00,-15.29,66.0,X-LC,4.32,329341.0,16453.0,155218.0,-0.35,5.26,47.13,54.87,3.0,0.11,2.29,13.98,X40,BTT,IT
42,ITC,452.00,-38.77,44.0,X-LC,0.87,198363.0,-1775.0,22673.0,-0.33,-0.89,11.43,10.44,4.0,-0.08,1.38,4.63,X40,NTT,FMCG
81,VBL,671.64,-19.14,45.0,X-LC,5.86,291375.0,-24467.0,137121.0,-0.57,-7.75,47.06,35.67,5.0,-0.18,2.02,5.03,X40N,ATH,FMCG
1,ABB,7934.00,-43.27,32.0,H-LC,5.15,237216.0,-24403.0,143611.0,0.37,-9.33,60.54,45.57,7.0,-0.17,1.65,0.94,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,311.06,81.0,L-MC,6.66,50416.0,-26790.0,52181.0,2.32,-34.70,103.50,32.89,258.0,-0.51,0.35,38.84,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,45.0,L-SC,22.25,77664.0,-35885.0,75940.0,0.00,-31.60,97.78,35.27,268.0,-0.47,0.54,90.08,XR,NTT,HOTELS
7,ASIANTILES,137.00,7588.89,66.0,L-SC,15.86,84368.0,-9442.0,86064.0,-1.21,-10.07,102.01,81.67,269.0,-0.11,0.59,63.38,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.15,49.0,H-SC,7.51,91680.0,-6300.0,27907.0,-0.78,-6.43,30.44,22.05,152.0,-0.23,0.64,33.26,XR,ATH,FINANCE
37,INDIAMART,4810.62,-52.59,61.0,H-SC,4.78,132558.0,9222.0,122404.0,-0.14,7.48,92.34,106.72,119.0,0.08,0.92,32.60,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,521.00,76.31,64.0,H-SC,7.67,153013.0,-29762.0,140833.0,2.42,-16.28,92.04,60.77,125.0,-0.21,1.06,37.80,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,0.01,85.0,X-LC,7.95,252296.0,528.0,52679.0,3.51,0.21,20.88,21.13,27.0,0.01,1.75,35.71,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,133.26,80.0,M-SC,4.36,152830.0,-21729.0,21763.0,2.49,-12.45,14.24,0.02,221.0,-1.00,1.06,32.94,OX40N,NTT,PAINTS
7,ASIANTILES,137.00,7588.89,66.0,L-SC,15.86,84368.0,-9442.0,86064.0,-1.21,-10.07,102.01,81.67,269.0,-0.11,0.59,63.38,XR,NTT,CERAMICS
14,BSOFT,836.99,-7.82,72.0,H-SC,9.02,103530.0,-37123.0,109907.0,2.24,-26.39,106.16,51.74,131.0,-0.34,0.72,19.78,XR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.83
1,25,44.55
2,50,76.27


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.53
LC    28.01
MC    27.06
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.45
X40      20.82
X40N     12.38
XY25     10.17
XR        9.80
AR        9.14
OX40N     6.55
X200      1.84
SR        1.05
MH        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.94
X-LC    20.00
M-SC    12.54
X-SC     5.60
H-LC     4.57
H-MC     3.77
M-LC     2.45
L-SC     1.46
L-LC     0.99
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.78,-4.21,37.91
IT,12.79,-14.22,75.35
FINANCE,12.07,-8.81,58.13
MISC,7.68,-17.67,77.03
PAINTS,5.96,-7.56,23.60
ELECTRICAL,5.60,-12.01,51.72
INSURANCE,4.50,2.55,31.79
PHARMA,3.48,0.26,33.20
AUTO,3.20,-51.23,113.83


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3215773.0,20
AR,1255567.0,10
XR,1247430.0,13
X40,972555.0,14
X40N,686509.0,9
OX40N,542131.0,9
XY25,471340.0,7
SR,269903.0,2
MH,62401.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3387632.0,24
X-MC,1383994.0,16
M-SC,1242804.0,15
X-LC,774466.0,11
X-SC,543972.0,6
H-MC,328571.0,3
H-LC,280569.0,3
L-SC,251065.0,3
M-LC,153949.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1173752.0      6
           AR         836971.0      5
           XR         782585.0      7
M-SC       XY24       726886.0      6
X-MC       XY24       556397.0      4
           X40        484429.0      8
X-LC       X40        401270.0      5
X-SC       X40N       289001.0      4
H-SC       SR         269903.0      2
           OX40N      262020.0      3
X-LC       X40N       239110.0      3
H-LC       AR         221542.0      2
X-MC       XY25       184770.0      2
H-MC       XY24       182014.0      1
X-SC       XY24       168115.0      1
L-SC       XR         162004.0      2
X-MC       X40N       158398.0      2
M-SC       XR         142452.0      2
           XY25       131919.0      1
           AR         122258.0      2
           OX40N      119289.0      4
M-LC       XR         108208.0      1
L-SC       OX40N       89061.0      1
X-SC       X40         86856.0      1
H-MC       AR          74796.0      1
X-LC       XY25        72291.0      2
H-MC       OX40N       71761.0      1
H-SC       MH          62401.0      1
X-LC       XY24        61795.0      1
H-LC       X200        59027.0      1
L-MC       XR          52181.0      1
M-LC       XY25        45741.0      1
L-LC       XY25        36619.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
